### Clonamos o diretório darknet

In [ ]:
!git clone https://github.com/AlexeyAB/darknet.git

Cloning into 'darknet'...
remote: Enumerating objects: 15833, done.
remote: Total 15833 (delta 0), reused 0 (delta 0), pack-reused 15833
Receiving objects: 100% (15833/15833), 14.39 MiB | 9.72 MiB/s, done.
Resolving deltas: 100% (10666/10666), done.


### entrando no diretório do darknet

In [ ]:
cd darknet

/content/darknet


### caso tenha uma GPU com suporte a CUDA abra o arquivo Makefile e mude as propriedades GPU=0 para 1, CUDNN=0 para 1, CUDNN_HALF=1 e compile o modelo usando o comando abaixo

In [ ]:
!make

mkdir -p ./obj/
mkdir -p backup
mkdir -p results
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -rdynamic -Ofast -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -rdynamic -Ofast -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/http_stream.cpp -o obj/http_stream.o
./src/http_stream.cpp: In member function ‘bool JSON_sender::write(const char*)’:
./src/http_stream.cpp:253:21: warning: unused variable ‘n’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-variable-Wunused-variable]8;;]
  253 |                 int n = _write(client, outputbuf, outlen);
      |                     ^
./src/http_stream.cpp: In function ‘void se

1. crie um arquivo myprog.cfg com o mesmo conteudo do arquivo yolov4-custom.cfg

2. mude o valor de max_batches para <numero de classes> * 2000

3. mude o valor de step para "80% do valor de max_batches" , "90% do valor de max_batches"

4. mude width e height para um valor multiplo de 32. Ex: 416

5. nos 3 blocos [convolutional] mude o valor de filters de 255 para (classes + 5)x3 e dos 3 blocos [yolo] o valor de classes para o seu numero de classes

6. crie um arquivo myprog.cfg em build\darknet\x64\data\ com o nome das classes, cada um por linha.

7. crie um arquivo myprog.data em build\darknet\x64\data\ configurando de acorde com sua necessidade no seguinte formato:<br>
   classes = (numero de classes)<br>
   train  = caminho/para/arquivo/train.txt<br>
   valid  = caminho/para/arquivo/test.txt<br>
   names = caminho/para/myprog.names<br>
   backup = backup/

8. coloque suas imagens em build\darknet\x64\data\myprog\ use uma ferramento como o labelImg ou o label-studio para criar o seu dataset rotulado de forma simples para o yolo.
Com uma dessas ferramentas vc tera as imagens acompanhadas de arquivos txt contendo as cordenadas da bounding box e a classe
no formato "object-class" "x_center" "y_center" "width" "height"

9. crie um arqui train.txt e test.txt em build\darknet\x64\data\ contendo os caminho completo para as imagens correspondentes

10. baixe os pesos do [yolo4.conv.137](https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137) e coloque em build\darknet\x64

### estando tudo configurado use o comando para iniciar o treinamento
obs: no meu computador (i5 2500k + 16RAM sem GPU com CUDA)o treinamento foi previsto para uma estimativa de quase 1 semanda então tenha em mente que dependendo da sua maquina pode demorar bastante.

In [ ]:
!./darknet detector train build/darknet/x64/data/myprog.data cfg/myprog.cfg build/darknet/x64/yolov4.conv.137

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.337856), count: 1, class_loss = 0.520817, iou_loss = 1.050428, total_loss = 1.571245 
 total_bbox = 11961, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 0.000042, iou_loss = 0.000000, total_loss = 0.000042 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.503027), count: 1, class_loss = 0.281963, iou_loss = 0.667371, total_loss = 0.949335 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.593424), count: 1, class_loss = 0.525090, iou_loss = 0.333692, total_loss = 0.858782 
 total_bbox = 11963, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 0.000155, iou_loss = 0.000000, total_loss = 0.000155 
v3 (mse

### Testando

In [ ]:
!./darknet detector map build/darknet/x64/data/myprog.data cfg/myprog.cfg build/darknet/x64/yolov4.conv.137

/content/darknet


### para usar o modelo treinado em uma imagem especifica use o seguinte comando:

In [ ]:
!python darknet_images.py --weights ./backup/myprog_last.weights \
                         --data_file build/darknet/x64/data/myprog.data \
                         --config_file cfg/myprog.cfg \
                         --input /caminho/para/img
